In [ ]:
##### DATA ####

homedir = '' # enter your homedir here
projectdir = homedir + '' # add projectdir

bamdir     = projectdir + 'ChIPseq/final_bam/dedup/' # you can download bamfiles from Zenodo: 10.5281/zenodo.1480328

name2bamfile = {}
name2bamfile['H3K4me2_a']  = bamdir + '1358_lane7-index13.mapped2.Fol2.bwa_aln.samtools_sort.picard_dedup.bam'
name2bamfile['H3K4me2_b']  = bamdir + 'HTS806_lane5-index0004.mapped2.Fol2.bwa_aln.samtools_sort.picard_dedup.bam'
name2bamfile['H3K27me3_a'] = bamdir + '1360_lane7-index15.mapped2.Fol2.bwa_aln.samtools_sort.picard_dedup.bam'
name2bamfile['H3K27me3_b'] = bamdir + '808_lane5-index0006.mapped2.Fol2.bwa_aln.samtools_sort.picard_dedup.bam'

name2bedfile = {}
name2bedfile['GENES']         = projectdir + 'GENES/fusarium_oxysporum_f._sp._lycopersici_4287_2_transcripts.gtf'
name2bedfile['REPEATS']       = projectdir + 'REPEATS/Fol4287_Repeats.bed' 
name2bedfile['SNPs_VCG030']   = projectdir + 'filteredSNPs/asBed/vcg030re.annot.pass.nodup.bed'
name2bedfile['SNPs_CladeIII'] = projectdir + 'filteredSNPs/asBed/cladeIIIvcg030re.annot.pass.nodup.bed'

value_per_gene_dir = projectdir + 'values_per_gene/'
name2gene2value = {}
name2gene2value['dNdS']  = value_per_gene_dir + 'dN_dS_values.filtered.perGene.tab'
name2gene2value['dN']    = value_per_gene_dir + 'dN_values.filtered.perGene.tab'
name2gene2value['dS']    = value_per_gene_dir + 'dS_values.filtered.perGene.tab'

name2gene2value['invitro_RPKM']   = value_per_gene_dir + 'in_vitro_RPKM.tab'
name2gene2value['inplanta_RPKM']  = value_per_gene_dir + 'in_planta_RPKM.tab'
name2gene2value['log2foldChange'] = value_per_gene_dir + 'log2foldChange.tab'

print('Chip-seq:')
for name in name2bamfile.keys():
    print(name, '\t', name2bamfile[name])
print('\n')
print('Gene- and repeat density:')
for name in name2bedfile.keys():
    print(name, '\t', name2bedfile[name])
print('\n')
print ('Value per gene:')
for name in name2gene2value.keys():
    print(name, '\t', name2gene2value[name])

## Correlations between genome characteristics

We have different characteristics mapped to the genome:

* gene-properties
    - relative expression level
    - dN
    - dS
    
* genome properties
    - read density ChIP-seq experiments
    - gene density
    - repeat density/copynumber

Some of these properties are interdependent, e.g. we may find H3K27Me3 in regions where gene are overexpressed in planta because this marker is instrumental in gene regulation. Similarly, we may find that regions that are enriched for H3K27Me3 are more divergent when compared to other species, because either genes are under less negative selection because they are not expressed except during infection, or because the de novo mutation rate is higher in these regions because they are less accessible to DNA repair machinery.

To answer these questions, we want to determine which characterics 'go together' *most strongly*. We divide the genome into windows, calculated average values for properties in these windows. Here, we calculate correlation coefficients T (Kendall's Tau because we can not assume that characteristics are normally distributed and thus can not use Pearson) between all characteristics and use these as distance measures (1-T) for clustering.

    

### 1. Density or average value per window

In [ ]:
#make one huge table with all values per window
codedir = homedir+'' #enter the folder where you cloned the repo

import sys
sys.path.append(codedir + 'tools/')
import window_tools

windows_fname  = projectdir + 'windows/Fol4287broad.10000_windows.bed'
wname = '10000'
contig2windows = {}
for line in open(windows_fname).readlines():
    contig, start, end = line.strip().split()
    if contig not in contig2windows: contig2windows[contig] = []
        
    window = start+'\t'+end
    contig2windows[contig].append(window)

    
# BAMFILES
all_window2values = {}
bamnames = list(name2bamfile.keys())
bamnames.sort()
names = []
for name in bamnames:
    contig2window2values = {}
    
    bam_fname = name2bamfile[name]
    density_fname = window_tools.density_bamfile(windows_fname, wname, bam_fname)
    for line in open(density_fname).readlines():
        contig, start, end, density = line.strip().split()
        if contig not in contig2window2values: contig2window2values[contig] = {}
        
        window = start+'\t'+end
        contig2window2values[contig][window] = density
        
    all_window2values[name] = contig2window2values
    names.append(name)

# BEDFILES
for name in name2bedfile.keys():
    #print(name)
    contig2window2values = {}
    
    bed_fname     = name2bedfile[name]
    density_fname = window_tools.density_bedfile(windows_fname, wname, bed_fname)
    #print(density_fname)
    if density_fname != None:
        for line in open(density_fname).readlines():
            contig, start, end, overlap = line.strip().split()
            if contig not in contig2window2values: contig2window2values[contig] = {}

            window = start+'\t'+end
            contig2window2values[contig][window] = overlap

        all_window2values[name] = contig2window2values
        names.append(name)

# FILES WITH A VALUE PER GENE
gene_values = list(name2gene2value.keys())
gene_values.sort()

scaffold2start2gene = window_tools.genebed2dict(projectdir + 'GENES/Fol4287__genes.bed')
window2genes        = window_tools.get_window2genes(windows_fname, scaffold2start2gene)
for name in gene_values:
    
    contig2window2values = {}
    gene2value_fname     = name2gene2value[name]
    density_fname        = window_tools.average_gene_values_per_window(windows_fname, wname, gene2value_fname, window2genes)
    for line in open(density_fname).readlines():
        data = line.split('\t')
        contig, start, end, mean = data[:4]
        if contig not in contig2window2values: contig2window2values[contig] = {}
        
        window = start+'\t'+end
        contig2window2values[contig][window] = mean
        
    all_window2values[name] = contig2window2values
    names.append(name)
print(names)


### 2. Write these density into a single file

Now we have a 'superdictionary' with all values (densities, averages of genes). We will write this to a file (to go as SOM with the paper), ordered according to the 3-speed genome: core, fast-core, accessory

In [ ]:
import plot_tools

contiglist = plot_tools.keyword_to_contigslist('Fol4287_3speed', prefix = 'Supercontig_2.', gapsize = 10)

# note that this list does not contain unpositioned scaffolds
# add unpositioned scaffolds at the end of the list (after a gap)
contiglist.append('__GAP')
contigset = set(contiglist)
for contig in set(contig2windows.keys()).difference(contigset):
    contiglist.append(contig)

all_values_per_window_fname = projectdir + 'windows/all_values_per_window.tab'
outfile = open(all_values_per_window_fname, 'w')
header  = 'Supercontig\tstart\tend'
#print(names)
for name in names:
    header += '\t'+name
outfile.write(header+'\n')
#print(header)
for contig in contiglist:
    #print(contig)
    if contig[:5] != '__GAP':
        
        for window in contig2windows[contig]:
            line = contig+'\t'+window
            for name in names:
                line += '\t'+all_window2values[name][contig][window]
            outfile.write(line+'\n')
        
outfile.close()

### 3. Clustering of values and of windows, heatmap

In [ ]:
# check versions of scipy (scipy 0.18 has a bug in the clustering algorithm)
import scipy
print(scipy.__version__)

In [ ]:
import numpy as np

def similarity_matrix_2D__to__distance_matrix_1D(matrix):
    distances = []
    for row_index, row in enumerate(matrix[:-1]):
        for value in row[row_index+1:]:
            distances.append(1-value)
            
    return distances

def TEST_similarity_matrix_2D__to__distance_matrix_1D():
    matrix = [[1,0.5,-0.3,0.4], [0.5,1,0.2,0.7],[-0.3,0.2, 1,-1.4],[0.4, 0.7, -1.4, 1]]
    
    print(similarity_matrix_2D__to__distance_matrix_1D(np.array(matrix)))
    
TEST_similarity_matrix_2D__to__distance_matrix_1D()
# must result in: [0.5, 1.3, 0.6, 0.8, 0.3, 2.4]

In [ ]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context("paper")

from scipy import stats
from scipy.spatial import distance
from scipy.cluster import hierarchy
from sklearn import preprocessing


windows2values = pd.read_csv(all_values_per_window_fname, sep = '\t', header = 0)

# Calculate correlations with P values for histone modifications and other characteristics
Hmods = ['H3K27me3_a', 'H3K27me3_b', 'H3K4me2_a', 'H3K4me2_b']
for Hmod in Hmods:
    print("_________________________________________________")
    print("Kendall's Tau for",Hmod,"with GENES")
    tau, Pvalue = stats.kendalltau(windows2values[Hmod], windows2values['GENES'])
    print(tau, Pvalue,'\n')
    
    print("Kendall's Tau for",Hmod,"with REPEATS")
    tau, Pvalue = stats.kendalltau(windows2values[Hmod], windows2values['REPEATS'])
    print(tau, Pvalue,'\n')
    
  


    

In [ ]:
# Correlation heatmap
windows2values = pd.read_csv(all_values_per_window_fname, sep = '\t', header = 0)
windows2values.drop('Supercontig', axis=1, inplace=True)
windows2values.drop('start', axis=1, inplace=True)
windows2values.drop('end', axis=1, inplace=True)

# Pandas deals with missing values correctly, hence we will use pandas to calculate correlations
# min_periods is minimum number of values in a column
corr_df           = windows2values.corr(method='spearman', min_periods = 20)
#print(corr_df)
similarity_matrix = corr_df.as_matrix()

#convert this similarity matrix into a distance matrix that is suitable for scipy.cluster.hierarchy
distance_matrix   = similarity_matrix_2D__to__distance_matrix_1D(similarity_matrix)

row_linkage = hierarchy.linkage(distance_matrix, method='average')
#print(row_linkage)

clst = sns.clustermap(similarity_matrix, row_linkage=row_linkage, col_linkage = row_linkage, vmin = -1.0, vmax = 1.0, cmap = 'RdBu')#RdYlGn')

# get correct labeling
name_index2name = {}
for name_index, name in enumerate(names):
    name_index2name[name_index] = name
    
xlabels_as_index = []
for textobject in clst.ax_heatmap.get_xticklabels():
    xlabels_as_index.append(int(textobject.get_text()))
print(xlabels_as_index)

xaxis_labels = []
for li in xlabels_as_index:
    xaxis_labels.append(name_index2name[li])
clst.ax_heatmap.set_xticklabels(xaxis_labels, rotation = 'vertical')

ylabels_as_index = []
for textobject in clst.ax_heatmap.get_yticklabels():
    ylabels_as_index.append(int(textobject.get_text()))
print(ylabels_as_index)

yaxis_labels = []
for li in ylabels_as_index:
    yaxis_labels.append(name_index2name[li])
clst.ax_heatmap.set_yticklabels(yaxis_labels, rotation = 'horizontal')
      
        
plt.savefig(projectdir + 'windows/HEATMAP_values_per_window__RdBu2.eps')#YlGn.eps')
plt.show()
                         

In [ ]:
# get list of scaffols per category
chr_categories = ['core', 'fast-core', 'accessory', 'lineage-specific', 'pathogenicity']
cat2chrs = {}

cat2chrs['core']             = ['chr01_C','chr02_C','chr04','chr05','chr07','chr08','chr09','chr10',]
cat2chrs['fast-core']        = ['chr11','chr12','chr13']
cat2chrs['accessory']        = ['chr01_LS', 'chr02_LS', 'chr03P', 'chr03A', 'chr06A','chr06P','chr15','chr14']
cat2chrs['lineage-specific'] = ['chr01_LS', 'chr02_LS', 'chr03A', 'chr06A','chr15']
cat2chrs['pathogenicity']    = ['chr14', 'chr03P', 'chr06P']

chr2scaffolds = {}
chr2scaffolds['chr01']    = [14, 1, 27]
chr2scaffolds['chr01_C']  = [14, 1]
chr2scaffolds['chr01_LS'] = [27]
chr2scaffolds['chr02']    = [6,10, 31]
chr2scaffolds['chr02_C']  = [6, 10]
chr2scaffolds['chr02_LS'] = [31]
chr2scaffolds['chr03']    = [47,18,32,7,25]
chr2scaffolds['chr03A']   = [32, 7, 25]
chr2scaffolds['chr03P']   = [47, 18]
chr2scaffolds['chr04']    = [8, 4]
chr2scaffolds['chr05']    = [26, 2]
chr2scaffolds['chr06']    = [9, 33, 41, 21, 53, 42]
chr2scaffolds['chr06A']   = [9, 33]
chr2scaffolds['chr06P']   = [41, 21, 53, 42]
chr2scaffolds['chr07']    = [5, 13]
chr2scaffolds['chr08']    = [3, 29]
chr2scaffolds['chr09']    = [11, 17]
chr2scaffolds['chr10']    = [20, 15, 45]
chr2scaffolds['chr11']    = [35, 12]
chr2scaffolds['chr12']    = [19, 23]
chr2scaffolds['chr13']    = [16, 39]
chr2scaffolds['chr14']    = [22, 43, 51, 36]
chr2scaffolds['chr15']    = [37, 38, 24, 28]


In [ ]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
sns.set_context("paper")
import matplotlib.pyplot as plt

from scipy import stats

from scipy.spatial import distance
from scipy.cluster import hierarchy
from sklearn import preprocessing


all_values_per_window_fname = projectdir + 'windows/all_values_per_window.tab'
allwindows2values = pd.read_csv(all_values_per_window_fname, sep = '\t', header = 0)


for cat in chr_categories:
    print(cat)
    subset_names = []+names
    #filter out windows located on supercontigs that are not in this category
    include = []
    for chr in cat2chrs[cat]:
        for scaff in chr2scaffolds[chr]:
            include.append('Supercontig_2.'+str(scaff))
            
    windows2values = allwindows2values[allwindows2values.Supercontig.isin(include)]
    windows2values.drop('Supercontig', axis=1, inplace=True)
    windows2values.drop('start', axis=1, inplace=True)
    windows2values.drop('end', axis=1, inplace=True)
    
    if cat == 'accessory' or cat == 'lineage-specific' or cat == 'pathogenicity':
        windows2values.drop('dN', axis=1, inplace=True)
        windows2values.drop('dS', axis=1, inplace=True)
        windows2values.drop('dNdS', axis=1, inplace=True)
        subset_names.remove('dN')
        subset_names.remove('dS')
        subset_names.remove('dNdS')
      

    # Pandas deals with missing values correctly, hence we will use pandas to calculate correlations
    corr_df           = windows2values.corr(method='spearman', min_periods = 20)
    similarity_matrix = corr_df.as_matrix()
    
    #convert this similarity matrix into a distance matrix that is suitable for scipy.cluster.hierarchy
    distance_matrix   = similarity_matrix_2D__to__distance_matrix_1D(similarity_matrix)

    row_linkage = hierarchy.linkage(distance_matrix, method='average')
    #print row_linkage

    clst = sns.clustermap(similarity_matrix, row_linkage=row_linkage, col_linkage = row_linkage, vmin = -1.0, vmax = 1.0, cmap = 'RdBu_r')

    # get correct labeling
    name_index2name = {}
    for name_index, name in enumerate(subset_names):      
        name_index2name[name_index] = name

    xlabels_as_index = []
    for textobject in clst.ax_heatmap.get_xticklabels():
        xlabels_as_index.append(int(textobject.get_text()))
    

    xaxis_labels = []
    for li in xlabels_as_index:
        xaxis_labels.append(name_index2name[li])
    clst.ax_heatmap.set_xticklabels(xaxis_labels, rotation = 'vertical')

    ylabels_as_index = []
    for textobject in clst.ax_heatmap.get_yticklabels():
        ylabels_as_index.append(int(textobject.get_text()))
    

    yaxis_labels = []
    for li in ylabels_as_index:
        yaxis_labels.append(name_index2name[li])
    clst.ax_heatmap.set_yticklabels(yaxis_labels, rotation = 'horizontal')


    plt.savefig(projectdir + 'windows/HEATMAP_'+cat+'_values_per_window__BuRd.eps')
    plt.show()
    



In [ ]:
windows2values = pd.read_csv(all_values_per_window_fname, sep = '\t', header = 0)
windows2values.drop('Supercontig', axis=1, inplace=True)
windows2values.drop('start', axis=1, inplace=True)
windows2values.drop('end', axis=1, inplace=True)

exclude = ['SNPs_VCG030', 'SNPs_CladeIII']
names.remove('SNPs_VCG030')
names.remove('SNPs_CladeIII')

for prop in exclude:
    windows2values.drop(prop, axis=1, inplace=True)
print(windows2values)
# Pandas deals with missing values correctly, hence we will use pandas tp calculate correlations
# min_periods is minimum number of values in a column
corr_df           = windows2values.corr(method='spearman', min_periods = 20)
print(corr_df)
similarity_matrix = corr_df.as_matrix()

#convert this similarity matrix into a distance matrix that is suitable for scipy.cluster.hierarchy
distance_matrix   = similarity_matrix_2D__to__distance_matrix_1D(similarity_matrix)

row_linkage = hierarchy.linkage(distance_matrix, method='average')
#print(row_linkage)

clst = sns.clustermap(similarity_matrix, row_linkage=row_linkage, col_linkage = row_linkage, vmin = -1.0, vmax = 1.0, cmap = 'RdBu')#RdYlGn')

# get correct labeling
name_index2name = {}
for name_index, name in enumerate(names):
    if name not in exclude:
        name_index2name[name_index] = name
    
xlabels_as_index = []
for textobject in clst.ax_heatmap.get_xticklabels():
    xlabels_as_index.append(int(textobject.get_text()))
print(xlabels_as_index)

xaxis_labels = []
for li in xlabels_as_index:
    xaxis_labels.append(name_index2name[li])
clst.ax_heatmap.set_xticklabels(xaxis_labels, rotation = 'vertical')

ylabels_as_index = []
for textobject in clst.ax_heatmap.get_yticklabels():
    ylabels_as_index.append(int(textobject.get_text()))
print(ylabels_as_index)

yaxis_labels = []
for li in ylabels_as_index:
    yaxis_labels.append(name_index2name[li])
clst.ax_heatmap.set_yticklabels(yaxis_labels, rotation = 'horizontal')
      
        
plt.savefig(projectdir + 'windows/HEATMAP_values_per_window__RdBu_noSNPs.eps')#YlGn.eps')
plt.show()